In [1]:
from random import uniform
import time
from IPython.display import display, clear_output
import numpy as np
import math
from copy import deepcopy
from collections import deque

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [2]:
# Mitu nuppu kummalgi mängijal laua läbinud on
PLAYER1_COMPLETED = 0 
PLAYER2_COMPLETED = 0

# Mis positsioonidel nupud mängus on
PLAYER1_POS_IN_PLAY = set() 
PLAYER2_POS_IN_PLAY = set()

# Defineerib võimalikud asukohad mõlema mängija jaoks
PLAYER1_TILES = {1 : [0,3], 2 : [0,2], 3 : [0,1], 4 : [0,0], 5 : [1,0], 6 : [1,1], 7 : [1,2], 8 : [1,3], 9 : [1,4], 10 : [1,5], 11 : [1,6], 12 : [1,7], 13 : [0,7], 14 : [0,6]} 
PLAYER2_TILES = {1 : [2,3], 2 : [2,2], 3 : [2,1], 4 : [2,0], 5 : [1,0], 6 : [1,1], 7 : [1,2], 8 : [1,3], 9 : [1,4], 10 : [1,5], 11 : [1,6], 12 : [1,7], 13 : [2,7], 14 : [2,6]} 

# Mis positsioonidel kumbki mängija sama käigu jooksul uuesti veeretada saab
ROLL_AGAIN_TILES = [4, 8, 14]

# Iga täringuveeretuse arvu tõenäosused
PROBABILITIES = {0 : 1 / 16, 1 : 4 / 16, 2 : 6 / 16, 3 : 4 / 16, 4 : 1 / 16}

# Mitme sammu võrra vaatab AI vastane edasi
MINMAX_DEPTH = 2

GAMESTATES = deque()

In [3]:
def increaseScore(player):
    global PLAYER1_COMPLETED, PLAYER2_COMPLETED
    if (player == 1):
        PLAYER1_COMPLETED += 1
    else:
        PLAYER2_COMPLETED += 1
    return

def decreaseScore(player):
    global PLAYER1_COMPLETED, PLAYER2_COMPLETED
    if (player == 1):
        PLAYER1_COMPLETED -= 1
    else:
        PLAYER2_COMPLETED -= 1
    return

In [4]:
def getScore(player):
    if (player == 1):
        return PLAYER1_COMPLETED
    else:
        return PLAYER2_COMPLETED

In [5]:
def resetGameState():
    global PLAYER1_COMPLETED, PLAYER2_COMPLETED, PLAYER1_POS_IN_PLAY, PLAYER2_POS_IN_PLAY

    PLAYER1_COMPLETED = 0 
    PLAYER2_COMPLETED = 0

    PLAYER1_POS_IN_PLAY = set() 
    PLAYER2_POS_IN_PLAY = set()

In [6]:
# Tagastab:
# 0 0 0 0     0 0
# 0 0 0 0 0 0 0 0
# 0 0 0 0     0 0
def initBoard():
    resetGameState()
    board = np.zeros(shape=(3,8)).astype(int).astype(str)
    board[0][4:6] = " "
    board[2][4:6] = " "
    return board

In [7]:
def prettyPrint(board):
    print("Skoor " + str(PLAYER1_COMPLETED) + " : " + str(PLAYER2_COMPLETED) + "\n")
    print('\n'.join([' '.join([str(cell) for cell in row]) for row in board]) + "\n")
prettyPrint(initBoard())

Skoor 0 : 0

0 0 0 0     0 0
0 0 0 0 0 0 0 0
0 0 0 0     0 0



In [8]:
def rollDice():
    result = 0
    for i in range(4):
        result += np.random.randint(2)
    return result

In [23]:
def movePiece(board, player, currentPosLocIdx, steps, debug=False):
    playerMovePath = PLAYER1_TILES     if player == 1 else PLAYER2_TILES
    piecesInPlay = PLAYER1_POS_IN_PLAY if player == 1 else PLAYER2_POS_IN_PLAY
    
    enemyPiecesInPlay = PLAYER2_POS_IN_PLAY if player == 1 else PLAYER1_POS_IN_PLAY  

    if (len(piecesInPlay) + getScore(player) == 7) and currentPosLocIdx == 0:
        if (debug):
            print("No more pieces to add to the board")
        return board, False

    if steps < 1:
        return board, False
    
    if steps > 4:
        if (debug):
            print("Not a valid dice roll result")
        return board, False

    if currentPosLocIdx not in piecesInPlay and currentPosLocIdx != 0:
        if (debug):
            print("There is no gamepiece on the given tile. For player " + str(player) + ", there are pieces in play on the following positions: " + str(piecesInPlay))
            prettyPrint(board)
        return board, False

    if (currentPosLocIdx + steps) in piecesInPlay:
        if (debug):
            print("There already is a gamepiece on the target tile. You can't step on your own piece.")
            prettyPrint(board)
        return board, False
    
    if currentPosLocIdx != 0:
        old_x, old_y = playerMovePath.get(currentPosLocIdx)

    if (currentPosLocIdx + steps) > 14:
        board[old_x][old_y] = str(0)
        piecesInPlay.remove(currentPosLocIdx)
        increaseScore(player)

        if (getScore(player) >= 7):
            if (debug):
                print("Mängija " + str(player) + " võitis!")

        return board, True

    new_x, new_y = playerMovePath.get(currentPosLocIdx + steps)

    if currentPosLocIdx != 0:
        board[old_x][old_y] = str(0)
    board[new_x][new_y] = str(player)
    if currentPosLocIdx != 0:
        piecesInPlay.remove(currentPosLocIdx)
    piecesInPlay.add(currentPosLocIdx + steps)

    if (currentPosLocIdx + steps) in enemyPiecesInPlay and (currentPosLocIdx + steps) > 4 and (currentPosLocIdx + steps) < 13:
        enemyPiecesInPlay.remove(currentPosLocIdx + steps)

    return board, True

In [10]:
def humanTurn(board, player, diceResult):
    turnSuccessful = False
    while (turnSuccessful != True):
        try:
            currentPosLocIdx = -1
            while not (currentPosLocIdx >= 0 and currentPosLocIdx < 15): 
                currentPosLocIdx = int(input("Enter position of piece you'd like to move (0 to add new piece to board)"))
                print(currentPosLocIdx)

            turnSuccessful = movePiece(board, player, currentPosLocIdx, diceResult, True)[1]
        except:
            print("Invalid input")
            continue

def evaluateBoard(board):
    # Arv nuppe, mis on lõppu jõudnud
    total = getScore(1) - getScore(2)
    
    # Arv nuppe, mis on ohutul ruudul
    total += len(PLAYER1_POS_IN_PLAY & set(ROLL_AGAIN_TILES)) * 0.5
    total -= len(PLAYER2_POS_IN_PLAY & set(ROLL_AGAIN_TILES)) * 0.5
    
    return total

def isGameOver():
    return getScore(1) >= 7 or getScore(2) >= 7
    
def getPossibleMoves(board, player, steps):
    playerMovePath = PLAYER1_TILES     if player == 1 else PLAYER2_TILES
    piecesInPlay = (PLAYER1_POS_IN_PLAY if player == 1 else PLAYER2_POS_IN_PLAY) | {0}

    if ((len(piecesInPlay) - 1) + getScore(player) == 7): # -1 to account for the 0 that isn't an actual piece on the board
        piecesInPlay.remove(0)
    
    if steps < 1 or steps > 4:
        return set()
    
    validMoves = set()
    
    for piece in piecesInPlay:
        if (piece + steps) in piecesInPlay:
            continue
            
        validMoves.add(piece)
        
    return validMoves

def saveGameState(board):
    global PLAYER1_COMPLETED, PLAYER2_COMPLETED, PLAYER1_POS_IN_PLAY, PLAYER2_POS_IN_PLAY, GAMESTATES
    GAMESTATES.append((deepcopy(board), PLAYER1_COMPLETED, PLAYER2_COMPLETED, deepcopy(PLAYER1_POS_IN_PLAY), deepcopy(PLAYER2_POS_IN_PLAY)))
    
def loadGameState():
    global PLAYER1_COMPLETED, PLAYER2_COMPLETED, GAMESTATES, PLAYER1_POS_IN_PLAY, PLAYER2_POS_IN_PLAY
    state = GAMESTATES.pop()
    PLAYER1_COMPLETED = state[1]
    PLAYER2_COMPLETED = state[2]
    PLAYER1_POS_IN_PLAY = state[3]
    PLAYER2_POS_IN_PLAY = state[4]
    return state[0]

    
def getBestMove(board, player, diceResult, depth):
    global GAMESTATES
    possibleMoves = getPossibleMoves(board, player, diceResult)
    bestMove = None
    bestEval = -math.inf if player == 1 else math.inf
    for possibleMove in possibleMoves:
        saveGameState(board)
        possibleBoard = movePiece(deepcopy(board), player, possibleMove, diceResult)[0]
        possibleEval = minMax(possibleBoard, 3 - player, None, depth-1)[1]
        board = loadGameState()
        if bestMove == None or (possibleEval > bestEval and player == 1 or possibleEval < bestEval and player == 2):
            bestMove = possibleMove
            bestEval = possibleEval
    return bestMove, bestEval

def minMax(board, player, diceResult=None, depth=MINMAX_DEPTH):
    global PROBABILITIES
    if depth == 0 or isGameOver():
         return None, evaluateBoard(board)
    else:
        if diceResult != None: # Täringuvise on teada, vali parim liigutus.
            return getBestMove(board, player, diceResult, depth)
        else: # Täringuvise on teadmata, hinnasta mänguseis kaalutud keskmisena
            averageEval = 0
            for diceResult in range(5): # Vaatame läbi kõik võimalikud silmade visked 0-4
                probability = PROBABILITIES[diceResult]
                bestEval = getBestMove(board, player, diceResult, depth)[1]
                averageEval += probability * bestEval
            return None, averageEval
            
            
def AITurn(board, player, diceResult):
    movePiece(board, player, minMax(board, player, diceResult)[0], diceResult)

In [26]:
def play(player1, player2):
    player = 1
    board = initBoard()
    while getScore(1) < 7 and getScore(2) < 7:
        diceResult = rollDice()

        playerColor = bcolors.OKGREEN if player == 1 else bcolors.OKBLUE  
        print(playerColor + "----- Turn start for player " + str(player) + " ----- \n" + bcolors.ENDC)
        print("Player 1 positions:", PLAYER1_POS_IN_PLAY)
        print("Player 2 positions:", PLAYER2_POS_IN_PLAY)
        print("Dice roll result: " + str(diceResult))

        prettyPrint(board)

        if (diceResult == 0):
            print("Skipped turn due to 0 dice roll")
            player = 3 - player
            continue
        
        if player == 1:
            if player1 == "human":
                humanTurn(board, player, diceResult)
            else:
                AITurn(board, player, diceResult)
        else:
            if player2 == "human":
                humanTurn(board, player, diceResult)
            else:
                AITurn(board, player, diceResult)

        print(playerColor + "----- Turn end for player " + str(player) + " ----- \n" + bcolors.ENDC)
        player = 3 - player

    print(bcolors.OKCYAN + "Final board" + bcolors.ENDC)
    prettyPrint(board)

In [33]:
play("AI", "AI")

----- Turn start for player 1 ----- 

Player 1 positions: set()
Player 2 positions: set()
Dice roll result: 2
Skoor 0 : 0

0 0 0 0     0 0
0 0 0 0 0 0 0 0
0 0 0 0     0 0

----- Turn end for player 1 ----- 

----- Turn start for player 2 ----- 

Player 1 positions: {2}
Player 2 positions: set()
Dice roll result: 2
Skoor 0 : 0

0 0 1 0     0 0
0 0 0 0 0 0 0 0
0 0 0 0     0 0

----- Turn end for player 2 ----- 

----- Turn start for player 1 ----- 

Player 1 positions: {2}
Player 2 positions: {2}
Dice roll result: 2
Skoor 0 : 0

0 0 1 0     0 0
0 0 0 0 0 0 0 0
0 0 2 0     0 0

----- Turn end for player 1 ----- 

----- Turn start for player 2 ----- 

Player 1 positions: {4}
Player 2 positions: {2}
Dice roll result: 2
Skoor 0 : 0

1 0 0 0     0 0
0 0 0 0 0 0 0 0
0 0 2 0     0 0

----- Turn end for player 2 ----- 

----- Turn start for player 1 ----- 

Player 1 positions: {4}
Player 2 positions: {4}
Dice roll result: 2
Skoor 0 : 0

1 0 0 0     0 0
0 0 0 0 0 0 0 0
2 0 0 0     0 0

----- Turn